In [ ]:
import rasterio
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from scipy.interpolate import griddata
from pathlib import Path

# ============================================================================
# FUNCTION 1: Convert TIF to dense 2km grid using interpolation
# ============================================================================
def tif_to_2km_grid(tif_path, variable_name='value', spacing_km=2):
    """
    Convert a GeoTIFF to a DataFrame with 2km spacing grid.
    Uses interpolation to create denser grid from original 5-minute data.
    
    Parameters:
    -----------
    tif_path : str
        Path to the GeoTIFF file
    variable_name : str
        Name for the value column
    spacing_km : float
        Desired spacing in kilometers (default 2km)
    
    Returns:
    --------
    pandas.DataFrame with coordinates every 2km
    """
    with rasterio.open(tif_path) as src:
        # Read original data
        data = src.read(1)
        transform = src.transform
        bounds = src.bounds  # (left, bottom, right, top)
        
        # Get original valid pixels
        rows, cols = np.where(~np.isnan(data))
        
        # Convert to geographic coordinates
        original_lons = []
        original_lats = []
        original_values = []
        
        for row, col in zip(rows, cols):
            lon, lat = rasterio.transform.xy(transform, row, col, offset='center')
            original_lons.append(lon)
            original_lats.append(lat)
            original_values.append(data[row, col])
        
        original_lons = np.array(original_lons)
        original_lats = np.array(original_lats)
        original_values = np.array(original_values)
        
        # Create new dense grid with 2km spacing
        # Convert km to degrees (approximate at Algeria/Tunisia latitude ~35°N)
        km_to_deg_lat = spacing_km / 111.0  # 1 degree latitude ≈ 111 km
        km_to_deg_lon = spacing_km / (111.0 * np.cos(np.radians(35)))  # Adjust for latitude
        
        # Create new grid
        new_lons = np.arange(bounds.left, bounds.right, km_to_deg_lon)
        new_lats = np.arange(bounds.bottom, bounds.top, km_to_deg_lat)
        
        # Create meshgrid
        grid_lon, grid_lat = np.meshgrid(new_lons, new_lats)
        
        print(f"Original grid: {len(original_lons)} points (~7-9 km spacing)")
        print(f"New grid: {grid_lon.size} points (~{spacing_km} km spacing)")
        
        # Interpolate values to new grid
        grid_values = griddata(
            points=(original_lons, original_lats),
            values=original_values,
            xi=(grid_lon, grid_lat),
            method='cubic',  # Can use 'linear', 'cubic', or 'nearest'
            fill_value=np.nan
        )
        
        # Flatten arrays and remove NaN values
        flat_lon = grid_lon.flatten()
        flat_lat = grid_lat.flatten()
        flat_values = grid_values.flatten()
        
        # Keep only valid values (within original data extent)
        valid_mask = ~np.isnan(flat_values)
        
        df = pd.DataFrame({
            'longitude': flat_lon[valid_mask],
            'latitude': flat_lat[valid_mask],
            variable_name: flat_values[valid_mask]
        })
        
        return df


# ============================================================================
# FUNCTION 2: Process multiple TIFs to 2km grid
# ============================================================================
def process_multiple_tifs_to_2km(tif_folder, spacing_km=2):
    """
    Process multiple TIF files and create a unified 2km spacing grid.
    
    Parameters:
    -----------
    tif_folder : str
        Path to folder containing TIF files
    spacing_km : float
        Desired spacing in kilometers
    
    Returns:
    --------
    pandas.DataFrame with all climate variables at 2km spacing
    """
    tif_folder = Path(tif_folder)
    tif_files = sorted(tif_folder.glob('*.tif'))
    
    if len(tif_files) == 0:
        raise ValueError(f"No TIF files found in {tif_folder}")
    
    print(f"Found {len(tif_files)} TIF files")
    print(f"Creating {spacing_km}km spacing grid...\n")
    
    # Process first file to establish grid
    first_file = tif_files[0]
    var_name = extract_variable_name(first_file.stem)
    print(f"Processing: {first_file.name} -> {var_name}")
    
    result_df = tif_to_2km_grid(str(first_file), var_name, spacing_km)
    
    # Process remaining files
    for tif_file in tif_files[1:]:
        var_name = extract_variable_name(tif_file.stem)
        print(f"\nProcessing: {tif_file.name} -> {var_name}")
        
        df = tif_to_2km_grid(str(tif_file), var_name, spacing_km)
        
        # Merge on coordinates (with small tolerance for floating point)
        result_df = result_df.merge(
            df,
            on=['longitude', 'latitude'],
            how='outer'
        )
    
    print(f"\n{'='*60}")
    print(f"Final DataFrame shape: {result_df.shape}")
    print(f"Grid spacing: ~{spacing_km} km")
    print(f"Total grid points: {len(result_df)}")
    print(f"{'='*60}")
    
    return result_df



def resample_tif_to_2km(tif_path, variable_name, output_resolution_km=2):
    """
    Resample a TIF to 2km resolution directly using rasterio.
    This is FASTER than the interpolation method above.
    
    Parameters:
    -----------
    tif_path : str
        Path to input TIF
    variable_name : str
        Variable name
    output_resolution_km : float
        Desired resolution in km
    
    Returns:
    --------
    pandas.DataFrame
    """
    from rasterio.enums import Resampling
    
    with rasterio.open(tif_path) as src:
        # Calculate new dimensions
        # Convert km to degrees
        km_to_deg = output_resolution_km / 111.0
        
        # Calculate scale factor
        original_res = src.res[0]  # Original resolution in degrees
        scale_factor = original_res / km_to_deg
        
        # New dimensions
        new_width = int(src.width * scale_factor)
        new_height = int(src.height * scale_factor)
        
        # Read and resample
        data = src.read(
            1,
            out_shape=(new_height, new_width),
            resampling=Resampling.bilinear
        )
        
        # Calculate new transform
        new_transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )
        
        # Extract coordinates
        rows, cols = np.where(~np.isnan(data))
        
        lons = []
        lats = []
        values = []
        
        for row, col in zip(rows, cols):
            lon, lat = rasterio.transform.xy(new_transform, row, col, offset='center')
            lons.append(lon)
            lats.append(lat)
            values.append(data[row, col])
        
        df = pd.DataFrame({
            'longitude': lons,
            'latitude': lats,
            variable_name: values
        })
        
        print(f"Resampled from {src.height}x{src.width} to {new_height}x{new_width}")
        print(f"Original resolution: ~{original_res * 111:.1f} km")
        print(f"New resolution: ~{output_resolution_km} km")
        print(f"Total points: {len(df)}")
        
        return df



def extract_variable_name(filename):
    parts = filename.replace('-', '_').split('_')
    for i, part in enumerate(parts):
        if part in ['tmin', 'tmax', 'prec']:
            variable = part
            if i + 1 < len(parts):
                date_parts = '_'.join(parts[i+1:])
                return f"{variable}_{date_parts}"
    return filename



if __name__ == "__main__":
    
   
    
    print("\n" + "="*60)
    print("Processing all TIFs to 2km grid")
    print("="*60)
    
    climate_2km_df = process_multiple_tifs_to_2km(
        r"D:\S3\data_mining\projet\clipped_imputed_data2/",
        spacing_km=2
    )
    
    climate_2km_df.to_csv('climate_2km_grid.csv', index=False)
    print("\nSaved to: climate_2km_grid.csv")
    
    climate_2km_gdf = gpd.GeoDataFrame(
        climate_2km_df,
        geometry=[Point(xy) for xy in zip(climate_2km_df.longitude, climate_2km_df.latitude)],
        crs="EPSG:4326"
    )
    
    climate_2km_gdf.to_file('climate_2km_grid.geojson', driver='GeoJSON')
    print("Saved to: climate_2km_grid.geojson")


Processing all TIFs to 2km grid
Found 12 TIF files
Creating 2km spacing grid...

Processing: wc2.1_cruts4.09_5m_tmin_2024-01_imputed_local.tif -> tmin_2024_01_imputed_local
Original grid: 37962 points (~7-9 km spacing)
New grid: 956920 points (~2 km spacing)


In [ ]:
import rasterio
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
import numpy as np
from scipy.interpolate import griddata
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')


def km_to_degrees(lat, spacing_km):
    
    km_per_deg_lat = 111.0
    km_per_deg_lon = 111.0 * np.cos(np.radians(lat))
    
    deg_spacing_lon = spacing_km / km_per_deg_lon
    deg_spacing_lat = spacing_km / km_per_deg_lat
    
    return deg_spacing_lon, deg_spacing_lat

def extract_month_from_filename(filename):
    
    name = Path(filename).stem.lower()
    
    month_map = {
        '01': 'jan', '02': 'feb', '03': 'mar', '04': 'apr',
        '05': 'may', '06': 'jun', '07': 'jul', '08': 'aug', 
        '09': 'sep', '10': 'oct', '11': 'nov', '12': 'dec'
    }
    
    if 'tmin' in name:
        var_type = 'tmin'
    elif 'tmax' in name:
        var_type = 'tmax'
    elif 'tavg' in name:
        var_type = 'tavg'
    elif 'prec' in name:
        var_type = 'prec'
    else:
        var_type = 'unknown'
    
    for month_num, month_name in month_map.items():
        if f'-{month_num}' in name or f'_{month_num}' in name:
            return f"{var_type}_{month_name}"
    
    return name.replace('-', '_').replace(' ', '_')


def tif_to_2km_grid(tif_path, variable_name=None, spacing_km=2, method='linear'):
   
    if variable_name is None:
        variable_name = extract_month_from_filename(tif_path)
    
    with rasterio.open(tif_path) as src:
        data = src.read(1)
        transform = src.transform
        bounds = src.bounds
        
        orig_res_deg = src.res[0]
        orig_res_km = orig_res_deg * 111.0
        print(f"   Original resolution: {orig_res_deg:.4f}° (~{orig_res_km:.1f} km)")
        
        if spacing_km < orig_res_km:
            print("   ⚠️  Note: Creating finer grid via interpolation")
        
        rows, cols = np.where(~np.isnan(data) & (data != src.nodata))
        
        if len(rows) == 0:
            raise ValueError("No valid data points found")
        
        original_lons, original_lats, original_values = [], [], []
        
        for row, col in zip(rows, cols):
            lon, lat = rasterio.transform.xy(transform, row, col, offset='center')
            original_lons.append(lon)
            original_lats.append(lat)
            original_values.append(data[row, col])
        
        original_lons = np.array(original_lons)
        original_lats = np.array(original_lats)
        original_values = np.array(original_values)
        
        deg_spacing_lon, deg_spacing_lat = km_to_degrees(30, spacing_km)
        
        new_lons = np.arange(bounds.left, bounds.right, deg_spacing_lon)
        new_lats = np.arange(bounds.bottom, bounds.top, deg_spacing_lat)
        
        # Create meshgrid for interpolation
        grid_lon, grid_lat = np.meshgrid(new_lons, new_lats)
        
        # Interpolate values to new grid
        grid_values = griddata(
            points=(original_lons, original_lats),
            values=original_values,
            xi=(grid_lon, grid_lat),
            method=method,
            fill_value=np.nan
        )
        
        # Flatten arrays and remove NaN values
        flat_lon = grid_lon.flatten()
        flat_lat = grid_lat.flatten()
        flat_values = grid_values.flatten()
        
        valid_mask = ~np.isnan(flat_values)
        
        df = pd.DataFrame({
            'longitude': flat_lon[valid_mask],
            'latitude': flat_lat[valid_mask],
            variable_name: flat_values[valid_mask]
        })
        
        print(f"   ✅ {len(df):,} points | Variable: {variable_name}")
        
        return df

def resample_tif_to_2km(tif_path, variable_name=None, output_resolution_km=2):
    """
    Resample TIF to target resolution using rasterio's built-in resampling.
    """
    from rasterio.enums import Resampling
    
    if variable_name is None:
        variable_name = extract_month_from_filename(tif_path)
    
    with rasterio.open(tif_path) as src:
        # Calculate original resolution
        orig_res_deg = src.res[0]
        orig_res_km = orig_res_deg * 111.0
        
        print(f"   Original resolution: {orig_res_deg:.4f}° (~{orig_res_km:.1f} km)")
        
        # Calculate degree spacing
        deg_spacing_lon, deg_spacing_lat = km_to_degrees(30, output_resolution_km)
        target_res_deg = min(deg_spacing_lon, deg_spacing_lat)
        
        # Calculate scale factor
        scale_factor = orig_res_deg / target_res_deg
        
        # New dimensions
        new_width = int(src.width * scale_factor)
        new_height = int(src.height * scale_factor)
        
        # Read and resample
        data = src.read(
            1,
            out_shape=(new_height, new_width),
            resampling=Resampling.bilinear
        )
        
        # Calculate new transform
        new_transform = src.transform * src.transform.scale(
            (src.width / new_width),
            (src.height / new_height)
        )
        
        # Extract coordinates and values
        rows, cols = np.where(~np.isnan(data) & (data != src.nodata))
        
        lons, lats, values = [], [], []
        
        for row, col in zip(rows, cols):
            lon, lat = rasterio.transform.xy(new_transform, row, col, offset='center')
            lons.append(lon)
            lats.append(lat)
            values.append(data[row, col])
        
        df = pd.DataFrame({
            'longitude': lons,
            'latitude': lats,
            variable_name: values
        })
        
        print(f"   ✅ {len(df):,} points | Variable: {variable_name}")
        
        return df

# ============================================================================
# BATCH PROCESSING - SIMPLIFIED
# ============================================================================

def process_temperature_data(tif_folder, spacing_km=2, method='resample'):
    """
    Process all temperature TIF files and create unified 2km grid.
    
    Parameters:
    -----------
    tif_folder : str
        Path to folder containing TIF files
    spacing_km : float
        Desired spacing in kilometers
    method : str
        'interpolate' or 'resample'
    
    Returns:
    --------
    pandas.DataFrame with all temperature variables at 2km spacing
    """
    tif_folder = Path(tif_folder)
    tif_files = sorted(tif_folder.glob('*.tif'))
    
    if len(tif_files) == 0:
        raise ValueError(f"❌ No TIF files found in {tif_folder}")
    
    print(f"📁 Found {len(tif_files)} TIF files")
    print(f"🎯 Creating {spacing_km}km spacing grid using {method} method...")
    print()
    
    # First, analyze the files
    print("📊 FILE ANALYSIS:")
    for tif_file in tif_files:
        var_name = extract_month_from_filename(tif_file)
        print(f"   📄 {tif_file.name} → {var_name}")
    
    print("\n🔄 PROCESSING FILES:")
    
    # Process all files and store in a list
    all_dataframes = []
    
    for i, tif_file in enumerate(tif_files, 1):
        var_name = extract_month_from_filename(tif_file)
        print(f"\n[{i}/{len(tif_files)}] {var_name}")
        
        try:
            if method == 'interpolate':
                df = tif_to_2km_grid(str(tif_file), var_name, spacing_km)
            else:
                df = resample_tif_to_2km(str(tif_file), var_name, spacing_km)
            
            # Round coordinates to avoid floating point issues
            df['longitude'] = df['longitude'].round(6)
            df['latitude'] = df['latitude'].round(6)
            
            all_dataframes.append(df)
            
        except Exception as e:
            print(f"   ❌ Failed: {str(e)}")
            continue
    
    if not all_dataframes:
        raise ValueError("❌ No files were successfully processed")
    
    print(f"\n🔄 MERGING {len(all_dataframes)} DATASETS...")
    
    # Start with first dataframe as base
    result_df = all_dataframes[0]
    
    # Merge all other dataframes
    for i, df in enumerate(all_dataframes[1:], 2):
        # Merge on coordinates
        before_count = len(result_df)
        result_df = result_df.merge(
            df,
            on=['longitude', 'latitude'],
            how='inner'
        )
        after_count = len(result_df)
        
        print(f"   Merge {i}: {before_count:,} → {after_count:,} points")
    
    # Final summary
    print(f"\n{'='*60}")
    print(f"📊 FINAL RESULTS:")
    print(f"   Grid spacing: ~{spacing_km} km")
    print(f"   Total grid points: {len(result_df):,}")
    print(f"   Temperature variables: {len(result_df.columns) - 2}")
    
    # List all temperature columns
    temp_columns = [col for col in result_df.columns if col not in ['longitude', 'latitude']]
    print(f"   Variables: {', '.join(sorted(temp_columns))}")
    print(f"{'='*60}")
    
    return result_df

# ============================================================================
# QUALITY CONTROL
# ============================================================================

def validate_temperature_grid(df):
    """
    Validate the temperature grid data.
    """
    print("\n🔍 DATA VALIDATION:")
    
    # Basic info
    print(f"   Grid points: {len(df):,}")
    print(f"   Columns: {len(df.columns)}")
    
    # Coordinate ranges
    lon_range = f"{df.longitude.min():.2f} to {df.longitude.max():.2f}"
    lat_range = f"{df.latitude.min():.2f} to {df.latitude.max():.2f}"
    print(f"   Longitude: {lon_range}")
    print(f"   Latitude: {lat_range}")
    
    # Temperature statistics
    temp_columns = [col for col in df.columns if col not in ['longitude', 'latitude']]
    for col in temp_columns:
        if col in df.columns:
            print(f"   {col}: {df[col].min():.1f} to {df[col].max():.1f} (mean: {df[col].mean():.1f})")
    
    # Check for missing values
    missing = df.isnull().sum().sum()
    print(f"   Missing values: {missing}")
    
    return True

# ============================================================================
# MAIN EXECUTION
# ============================================================================

if __name__ == "__main__":
    
    # Configuration
    TIF_FOLDER = r"D:\S3\data_mining\projet\percepitation_climate\precipitations"
    OUTPUT_CSV = "temperature_percipitation_2km_grid.csv"
    OUTPUT_GEOJSON = "temperature_percepitation_2km_grid.geojson"
    SPACING_KM = 2
    METHOD = "resample"  # "interpolate" or "resample"
    
    print("🌡️  TEMPERATURE DATA TO 2KM GRID CONVERSION")
    print("=" * 60)
    
    try:
        # Process all temperature files
        temperature_df = process_temperature_data(
            TIF_FOLDER,
            spacing_km=SPACING_KM,
            method=METHOD
        )
        
        # Validate the grid
        validate_temperature_grid(temperature_df)
        
        # Save as CSV
        temperature_df.to_csv(OUTPUT_CSV, index=False)
        print(f"\n💾 CSV saved to: {OUTPUT_CSV}")
        
        # Save as GeoJSON (optional)
        geometry = [Point(xy) for xy in zip(temperature_df.longitude, temperature_df.latitude)]
        temperature_gdf = gpd.GeoDataFrame(
            temperature_df,
            geometry=geometry,
            crs="EPSG:4326"
        )
        
        temperature_gdf.to_file(OUTPUT_GEOJSON, driver='GeoJSON')
        print(f"🗺️  GeoJSON saved to: {OUTPUT_GEOJSON}")
        
        # Final summary
        print(f"\n🎉 PROCESSING COMPLETE!")
        print(f"   Final dataset: {len(temperature_df):,} points")
        print(f"   Variables: {len(temperature_df.columns) - 2} temperature columns")
        
    except Exception as e:
        print(f"❌ ERROR: {str(e)}")
        import traceback
        traceback.print_exc()

🌡️  TEMPERATURE DATA TO 2KM GRID CONVERSION
📁 Found 12 TIF files
🎯 Creating 2km spacing grid using resample method...

📊 FILE ANALYSIS:
   📄 wc2.1_cruts4.09_5m_prec_2024-01.tif → prec_jan
   📄 wc2.1_cruts4.09_5m_prec_2024-02.tif → prec_feb
   📄 wc2.1_cruts4.09_5m_prec_2024-03.tif → prec_mar
   📄 wc2.1_cruts4.09_5m_prec_2024-04.tif → prec_apr
   📄 wc2.1_cruts4.09_5m_prec_2024-05.tif → prec_may
   📄 wc2.1_cruts4.09_5m_prec_2024-06.tif → prec_jun
   📄 wc2.1_cruts4.09_5m_prec_2024-07.tif → prec_jul
   📄 wc2.1_cruts4.09_5m_prec_2024-08.tif → prec_aug
   📄 wc2.1_cruts4.09_5m_prec_2024-09.tif → prec_sep
   📄 wc2.1_cruts4.09_5m_prec_2024-10.tif → prec_oct
   📄 wc2.1_cruts4.09_5m_prec_2024-11.tif → prec_nov
   📄 wc2.1_cruts4.09_5m_prec_2024-12.tif → prec_dec

🔄 PROCESSING FILES:

[1/12] prec_jan
   Original resolution: 0.0833° (~9.3 km)
   ✅ 698,106 points | Variable: prec_jan

[2/12] prec_feb
   Original resolution: 0.0833° (~9.3 km)
   ✅ 698,106 points | Variable: prec_feb

[3/12] prec_mar
  

In [3]:
import pandas as pd

# Simple one-liner merge
tmin = pd.read_csv(r"D:\S3\data_mining\projet2\temperature_2km_grid.csv")
tmax = pd.read_csv(r"D:\S3\data_mining\projet2\temperature_max_2km_grid.csv")
prec = pd.read_csv(r"D:\S3\data_mining\projet2\temperature_percipitation_2km_grid.csv")

# Round coordinates
for df in [tmin, tmax, prec]:
    df['longitude'] = df['longitude'].round(6)
    df['latitude'] = df['latitude'].round(6)

# Chain the merges
complete_climate = tmin.merge(tmax, on=['longitude', 'latitude'])\
                      .merge(prec, on=['longitude', 'latitude'])

print(f"✅ Merged {len(complete_climate):,} points")
complete_climate.to_csv("complete_climate.csv", index=False)

✅ Merged 698,106 points


In [4]:
complete_climate["is_fire"] = 0
complete_climate.head()

,longitude,latitude,tmin_jan,tmin_feb,tmin_mar,tmin_apr,tmin_may,tmin_jun,tmin_jul,tmin_aug,...,prec_apr,prec_may,prec_jun,prec_jul,prec_aug,prec_sep,prec_oct,prec_nov,prec_dec,is_fire
0,9.558559,37.32432,8.449325,8.5,9.449325,11.199325,15.0,18.449324,22.0,22.0,...,33.357433,20.113176,4.294932,1.025000,6.980405,33.809800,55.512160,40.542570,92.953040,0
1,9.576577,37.32432,8.395270,8.5,9.395270,11.145270,15.0,18.395270,22.0,22.0,...,33.152027,19.967230,4.289527,1.025000,7.012838,33.793583,55.685135,40.347973,92.396286,0
2,9.594595,37.32432,8.341216,8.5,9.341216,11.091216,15.0,18.341217,22.0,22.0,...,32.946620,19.821283,4.284122,1.025000,7.045270,33.777367,55.858110,40.153378,91.839530,0
3,9.612613,37.32432,8.287162,8.5,9.287162,11.037162,15.0,18.287163,22.0,22.0,...,32.741220,19.675339,4.278716,1.025000,7.077703,33.761150,56.031082,39.958786,91.282770,0
4,9.630631,37.32432,8.250000,8.5,9.233109,11.000000,15.0,18.233109,22.0,22.0,...,32.515540,19.551352,4.268243,1.036824,7.125338,33.780407,56.204056,39.833447,90.737840,0


In [5]:
fires = pd.read_csv(r"D:\S3\data_mining\projet\clean_data\fire_alg_tunis.csv")

In [6]:
fires['is_fire_fire'] = 1

In [7]:
import pandas as pd
import numpy as np

# SIMPLEST POSSIBLE VERSION
def simple_fire_merge(climate_file, fire_file, output_file, max_distance_km=2):
    """
    Ultra-simple fire-climate merge
    """
    # Load files
    climate = pd.read_csv(climate_file)
    fires = pd.read_csv(fire_file)
    
    # Find coordinate columns
    clon = [c for c in climate.columns if 'lon' in c.lower()][0]
    clat = [c for c in climate.columns if 'lat' in c.lower()][0]
    flon = [c for c in fires.columns if 'lon' in c.lower()][0] 
    flat = [c for c in fires.columns if 'lat' in c.lower()][0]
    
    # Convert km to degrees (approximate)
    max_distance_deg = max_distance_km * 0.009
    
    # Start with all points as non-fire
    climate['is_fire'] = 0
    
    # For each fire, find nearby climate points
    for _, fire in fires.iterrows():
        # Calculate distance to all climate points
        distance = np.sqrt(
            (climate[clat] - fire[flat])**2 + 
            (climate[clon] - fire[flon])**2
        )
        
        # Mark nearby points as fire
        climate.loc[distance <= max_distance_deg, 'is_fire'] = 1
    
    # Save result
    climate.to_csv(output_file, index=False)
    print(f"✅ Merged {len(fires)} fire points with {len(climate)} climate points")
    print(f"✅ Found {climate['is_fire'].sum()} fire-affected grid points")
    print(f"✅ Saved to: {output_file}")
    
    return climate

# USAGE:
if __name__ == "__main__":
    simple_fire_merge(
        climate_file="D:\S3\data_mining\projet2\complete_climate.csv",
        fire_file=r"D:\S3\data_mining\projet\clean_data\fire_alg_tunis.csv", 
        output_file="final_dataset.csv",
        max_distance_km=2
    )


✅ Merged 12924 fire points with 698106 climate points
✅ Found 6353 fire-affected grid points
✅ Saved to: final_dataset.csv


In [16]:
climate_df = pd.read_csv(r"D:\S3\data_mining\projet2\temperature_2km_grid.csv")
fires = pd.read_csv(r"D:\S3\data_mining\projet\clean_data\fire_alg_tunis.csv")

fire_climate_df = climate_df.merge(
    fires[['longitude','latitude',  'is_fire']],
    left_on=['longitude' , 'latitude' ],           # Columns in pivot_df (climate data)
    right_on=['longitude' ,'latitude'], # Columns in fires DataFrame
    how='left'
)


In [17]:
fire_climate_df.head()

,longitude,latitude,tmin_jan,tmin_feb,tmin_mar,tmin_apr,tmin_may,tmin_jun,tmin_jul,tmin_aug,tmin_sep,tmin_oct,tmin_nov,tmin_dec,is_fire
0,3.378378,37.32432,8.25,8.25,10.005068,11.510135,15.0,18.005068,21.010136,22.0,18.255068,16.755068,13.25,6.760135,NaN
1,3.396396,37.32432,8.25,8.25,10.032095,11.564189,15.0,18.032095,21.064190,22.0,18.282095,16.782095,13.25,6.814189,NaN
2,3.414414,37.32432,8.25,8.25,10.059122,11.618243,15.0,18.059122,21.118244,22.0,18.309122,16.809122,13.25,6.868243,NaN
3,3.432432,37.32432,8.25,8.25,10.086148,11.672298,15.0,18.086150,21.172297,22.0,18.336150,16.836150,13.25,6.922297,NaN
4,3.450450,37.32432,8.25,8.25,10.113175,11.726352,15.0,18.113176,21.226350,22.0,18.363176,16.863176,13.25,6.976351,NaN


In [ ]:
fire_climate_df.shape
(fire_climate_df['is_fire'] == 1).sum()

np.int64(0)

In [ ]:
climate_fire = represent_all_fires(r"D:\S3\data_mining\projet2\temperature_2km_grid.csv" , r"D:\S3\data_mining\projet\clean_data\fire_alg_tunis.csv" , "fire_climate1.csv")

🔥 REPRESENT ALL FIRES:
   Total fires: 12,924
   Successfully assigned: 12,924
   Unassigned fires: 0
   Fire-affected grid points: 2,582
   Assignment rate: 100.0%


In [8]:
climate_fire = pd.read_csv(r"D:\S3\data_mining\projet2\final_dataset.csv")

In [9]:
climate_fire.shape

(698106, 39)

In [10]:
(climate_fire["is_fire"] == 1).sum()

np.int64(6353)